In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 


In [2]:
class neuron:
    
    def __init__(self, n_inputs, activation, Lr = 0.01):
        self.w = np.random.randn(n_inputs, 1)
        self.b = np.zeros((1, 1))
        self.lose =[] #for viz
        self.activation = activation
        self.Lr = Lr
        self.n_inputs = n_inputs


    #activation functions
    def sigmoid(self, z):
        return 1 / (1 + np.exp(-np.clip(z, -250, 250)))
    

    def relu(self, z):
        return np.maximum(0, z)
    

    def tanh(self, z):
        return np.tanh(z) 
    

    #forward path to activate the neuron
    def forward(self, X):

        self.X = X
        z = X @ self.w + self.b

        #calling activation functions to activate the neuron to produce output(prediction)
        if self.activation == "Linear":
            self.y_pred = z
        elif self.activation == "Relu":
            self.y_pred = self.relu(z)
        elif self.activation == "Tanh":
            self.y_pred = self.tanh(z)
        elif self.activation == "Sigmoid":
            self.y_pred = self.sigmoid(z)


        return self.y_pred
    


    def loss(self, los, y_pred, Y_true):
        if self.los == "MSE":
            L = (Y_true - y_pred) ** 2
        elif self.los == "BCE":
            L = -[Y_true * np.log(y_pred) + (1-Y_true) * np.log(1-y_pred)]
        
        return L



    # backword --> calculate gradients
    def backword(self, los, y_pred, Y_true):

        #loss function first
        L = self.loss(los, y_pred, Y_true)
        #filling the loss list for viz
        self.lose.append(L)
        
        # global deriviatives
        dL_dy = 0
        if self.loss == "MSE":
            dy_dz = 1
            dL_dy = 2 * (Y_true - y_pred) * dy_dz
        elif self.loss == "BCE":
            dL_dy = (-(Y_true / y_pred + (1- Y_true)/(1-y_pred))) * dy_dz  # this is temporar till I find the correct answer

        
        dL_dw = dL_dy @ self.dw
        dL_db = np.sum(dL_dy) * self.db
        #dL_dx = self.dL_dy * self.dx

        return dL_dw, dL_db

    def step(self, dL_dw, dL_db):
        #update parameters
        self.w = self.w - self.Lr * dL_dw
        self.b = self.b - self.Lr * dL_db

        return self.w, self.b



In [ ]:
class DesnsLayer:
#adding the w initialization
    #n_output mean the number of neurons per layer
    def __init__(self, n_inputs, n_outputs, activation, los, Lr = 0.01, regularization='none', lambda_val=0.01, dropout_rate=0.0):
        self.n_inputs = n_inputs
        self.n_outputs = n_outputs
        self.activation = activation
        #self.w = np.random.randn(n_inputs, n_outputs) * 0.1 # here we multiplied by 0.1 to avoid the explding of the gradients because if the values of w came large then z will be big
        self.w = self.initialize_parameters(self.n_inputs, self.n_outputs, self.activation)
        self.b = np.zeros((1, n_outputs))
        self.Lr = Lr
        self.los = los 
        self.lose =[] #for viz
        self.lambda_val = lambda_val 
        self.regularization = regularization


    def initialize_parameters(self, n_inputs, n_outputs, activation):
            self.activation = activation
            self.n_inputs = n_inputs
            self.n_outputs = n_outputs
            if self.activation == 'Sigmoid' or self.activation == 'Tanh' :
                scale = np.sqrt(2.0 / (self.n_inputs + self.n_outputs))
                np.random.randn(self.n_inputs, n_outputs) * scale
                return np.random.randn(self.n_inputs, self.n_outputs) * 0.1
            elif self.activation == 'Relu':
                scale = np.sqrt(2.0 / self.n_inputs)
                np.random.randn(self.n_inputs) * scale
                return np.random.randn(self.n_inputs, self.n_outputs) * 0.1
    
    #activation functions
    def sigmoid(self, X):
        return 1 / (1 + np.exp(-np.clip(X, -250, 250)))
    
    def relu(self, X):
        return np.maximum(0, X)
    
    def tanh(self, X):
        return np.tanh(X)

    def forward(self, X):
        self.X = X
        self.z = X @ self.w + self.b
        #print(z)

        #calling activation functions to activate the neuron to produce output(prediction)
        if self.activation == "Linear":
            self.y_pred = self.z
        elif self.activation == "Relu":
            self.y_pred = self.relu(self.z)
        elif self.activation == "Tanh":
            self.y_pred = self.tanh(self.z)
        elif self.activation == "Sigmoid":
            self.y_pred = self.sigmoid(self.z)

        return self.y_pred


    def loss(self, los, y_pred, Y_true):
        dL_dy = 0
        if los == "MSE":
            L = np.mean((Y_true - y_pred) ** 2)
            self.lose.append(L)
            n = Y_true.shape[0]
            dL_dy = (2 /n) * (Y_true - y_pred)
        elif los == "BCE":
            # use the clip
            epsilon = 1e-15
            y_pred = np.clip(y_pred, epsilon, 1 - epsilon) #to avoid log(0)
            L = - np.mean(Y_true * np.log(y_pred) + (1 - Y_true) * np.log(1 - y_pred))
            self.lose.append(L)
            dL_dy = (y_pred - Y_true) / Y_true.shape[0]
            #(y_pred * (1- y_pred) * len(Y_true))
        
        return dL_dy


    # backword --> calculate gradients
    def backword(self, dL_dy):

        # activations derivatieves
        dy_dz = 0
        if self.activation == "Linear":
            dy_dz = 1
        elif self.activation == "Relu":
            dy_dz = (self.z > 0).astype(float)
        elif self.activation == "Tanh":
            dy_dz = 1 - self.y_pred**2
        elif self.activation == "Sigmoid":
            dy_dz = self.y_pred * (1 - self.y_pred)
        
        dL_dz = dL_dy * dy_dz
        dL_dw = self.X.T @ dL_dz
        dL_db = np.sum(dL_dz, axis=0, keepdims=1) * 1
        dL_dx = dL_dz @ self.w.T

         # Add regularization gradient to enhance dl_dw to avoid overfitting / the gradient of the summation and abs is the sign
        if self.regularization == 'L2':
            dL_dw = dL_dw + ((self.lambda_val / self.X.shape[0]) * self.w)
        elif self.regularization == 'L1':
            dL_dw = dL_dw + ((self.lambda_val / self.X.shape[0]) * np.sign(self.w))
            

        return dL_dw, dL_db, dL_dx

    def step(self, dL_dw, dL_db):
        #update parameters
        self.w = self.w - self.Lr * dL_dw
        self.b = self.b - self.Lr * dL_db

        return self.w, self.b
 

    def viz(self):
        plt.figure(figsize=(10, 6))
        plt.title("🧠 Neural Layer Architecture", fontsize=14, pad=20)

        # Draw neurons
        for i in range(self.n_inputs):  # Input layer
            plt.scatter(0, i, s=500, c='lightblue', edgecolors='black', zorder=5)
            plt.text(0, i, f'x{i+1}', ha='center', va='center', fontweight='bold')
            
        for j in range(self.n_outputs):  # Output layer
            plt.scatter(2, j, s=500, c='lightgreen', edgecolors='black', zorder=5)
            plt.text(2, j, f'a{j+1}', ha='center', va='center', fontweight='bold')
            
            # Draw connections
            for i in range(self.n_inputs):
                plt.plot([0, 2], [i, j], 'gray', alpha=0.3)

        plt.xlim(-0.5, 2.5)
        plt.ylim(-0.5, 4.5)
        plt.axis('off')
        plt.text(0, -0.3, f"Input Layer {self.n_inputs}, neurons", ha='center', fontsize=12)
        plt.text(2, -0.3, f"Dense Layer {self.n_outputs}, neurons", ha='center', fontsize=12)
        plt.show()



In [ ]:
#model building stack of layers MLP
class MLP:
#dropout_rate=0.0
    def __init__(self, n_neuorns_per_layer, list_activations=None, Lr = 0.01,  regularization='none', lambda_val=0.01):
    
        #item in zero index is the intput it self
        self.n_neuorns_per_layer = n_neuorns_per_layer
        self.list_activations = list_activations or ["Relu"] * (len(n_neuorns_per_layer) - 2) + ["Sigmoid"]
        #to stack layers in it
        self.layers = [] 
        self.lose = []
        self.lambda_val = lambda_val 
        self.regularization = regularization
        self.Lr = Lr
        
        #for the first layer is X it self and it is not counted in the loop as a layer, I see it as a layer but the program doesn't, it see the second layer
        # we said - 1 to skip first layer as it is the input
        for i in range(len(n_neuorns_per_layer) - 1): 
            self.layer = DesnsLayer(self.n_neuorns_per_layer[i], self.n_neuorns_per_layer[i + 1], self.list_activations[i], self.Lr, self.regularization, self.lambda_val)
            #creating a list of objects to call it later to access the class functions
            self.layers.append(self.layer)

    def forward(self, X):
        #to save the input to the memeory to know it later
        self.X = X 

        #because for each layer its input is the output of previous layer
        self.previous_output = X 
        for layer in self.layers:
            self.previous_output = self.layer.forward(self.previous_output)
        #return the last output as the inner ones are not important to me
        return self.previous_output  
    
    def backward(self, dL_dy):
        y_pred = self.previous_output

        parameters_gradient =[]
        current_grad = dL_dy
        for layer in reversed(self.layers):
            dL_dw, dL_db, dL_dx = self.layer.backword(current_grad)
            parameters_gradient.append((dL_dw, dL_db))
            current_grad = dL_dx

        return list(reversed(parameters_gradient))
    
    def step(self, parameters_gradient, Lr=0.01):
        for i, (dL_dw, dL_db) in enumerate(parameters_gradient):
            self.layers[i].w -= Lr * dL_dw
            self.layers[i].b -= Lr * dL_db


Dropout:

Deep Learning Connection: By randomly "banning" (dropping) neurons during training, we force the network to be robust. It cannot rely on just one specific input or pathway. It spreads the knowledge across all neurons.

In [ ]:
#building the classiifer
class loss_functions:
    def __init__(self, Y_true, y_pred):
        self.Y_true = Y_true
        self.y_pred = y_pred

    def MSE(self, Y_true, y_pred):
        L = np.mean((Y_true - y_pred) ** 2)
        return L

    def MSE_grad(self, Y_true, y_pred):
        n = Y_true.shape[0]
        L_grad = (2 /n) * (Y_true - y_pred)
        return L_grad

    def binary_cross_entropy(self, y_pred, y_true):
        epsilon = 1e-15
        y_pred = np.clip(y_pred, epsilon, 1 - epsilon) #to avoid log(0)
        L = - np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))
        return L
    
    #its gradiant
    def binary_cross_entropy_grad(self, y_pred, y_true):
        epsilon = 1e-15
        y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
        L = (y_pred - y_true) / (y_pred * (1 - y_pred) * len(y_true))
        return L
    

class NeruronNetworkLearning:

    def __init__(self, layer_size, los,  activations = None, Lr = 0.01, batch_size = 32, regularization='none', lambda_val=0.01, dropout_rate=0.0):
        self.mlp = MLP(layer_size, activations, regularization, lambda_val)
        self.layer_size = layer_size
        self.activations = activations
        self.batch_size = batch_size
        self.Lr = Lr
        self.loss_history = []
        self.accuracy_history = []
        self.eval_loss_history = []
        self.eval_acc_history = []
        self.lambda_val = lambda_val 
        self.regularization = regularization
        self.los = los

######### Regularaiation ########    
    def reg_loss(self, loss, reg_x):
        self.reg_x = reg_x
        #calcualte the weights of each layer
        reg_w = 0.0
        for layer in self.mlp.layers:
            if self.regularization == 'L2':
                reg_w += np.sum(layer.w**2)
            elif self.regularization == 'L1':
                reg_w += np.sum(np.abs(layer.w))
   
        if self.regularization == 'L2':
            #explain the formula in the markdown
            reg_loss = loss + ((self.lambda_val / (2 * self.reg_x[0])) * (reg_w))
        elif self.regularization == 'L1':
            reg_loss = loss + ((self.lambda_val / self.reg_x[0]) * (reg_w))
        else:
            reg_loss = loss
        
        return reg_loss

########## Training the data by looping through epochs and dividing it to batches for initial training
    def train(self, X_train, y_train, x_val, y_val, epochs = 1000, verbose = True):
        self.X_train = X_train
        n_samples = X_train.shape[0]
        self.y_train = y_train
        self.los_object = loss_functions(self.y_train, self.mlp.layer.y_pred)

        for epoch in range(epochs):

             #to avoid learning one type of output, it will memoeries leading to overtting, also we didn't use suffle function because it will break the dataset sequance
            index = np.random.permutation(n_samples)
            x_shaffeled = X_train[index]
            y_shaffeled = y_train[index]

            #loop to create batchs and train them
            for start_idx in range(0, n_samples, self.batch_size):
                end_index = min(start_idx + self.batch_size, n_samples)
                #slizing the dataset based on the batch size
                x_batch = x_shaffeled[start_idx : end_index]
                y_batch = y_shaffeled[start_idx: end_index]

                y_predd = self.mlp.forward(x_batch)

                dL_dy = 0.0
                if self.los == "MSE":
                    dL_dy = self.los_object.MSE_grad(y_predd, y_batch)
                elif self.los == "BCE":
                    dL_dy = self.los_object.binary_cross_entropy_grad(y_predd, y_batch)
                #dL_dy = self.los_object.binary_cross_entropy_grad(y_predd, y_batch)

                gradiants = self.mlp.backward(dL_dy)
                
                self.mlp.step(gradiants, self.Lr)
            
            ### Rerunning the whole data on the network to check if it learned
            y_pred_full_train = self.mlp.forward(X_train)
            full_train_loss1 = 0.0
            if self.los == "MSE":
                full_train_loss1 = self.los_object.MSE(y_predd, y_train)
            elif self.los == "BCE":
                full_train_loss1 = self.los_object.binary_cross_entropy(y_predd, y_train)
            ### Adding the regularaization to the loss
            #full_train_loss = self.binary_cross_entropy(y_pred_full_train, y_train)
            full_train_loss = self.reg_loss(full_train_loss1, X_train)
            full_train_acc = self.accuracy(y_pred_full_train, y_train)

            self.loss_history.append(full_train_loss)
            self.accuracy_history.append(full_train_acc)

            #validating the model larning on new
            if x_val is not None and y_val is not None:
                y_pred_val = self.mlp.forward(x_val)
                val_loss = 0.0
                if self.los == "MSE":
                    val_loss = self.los_object.MSE(y_predd, y_train, x_val)
                elif self.los == "BCE":
                    val_loss = self.los_object.binary_cross_entropy(y_predd, y_train)
                #val_loss = self.binary_cross_entropy(y_pred_val, y_val)
                val_loss = self.reg_loss(val_loss, x_val)
                val_acc = self.accuracy(y_pred_val, y_val)

                self.eval_loss_history.append(val_loss)
                self.eval_acc_history.append(val_acc)

            if verbose and (epoch % 100 == 0 or epoch == epochs - 1):
                val_info = f" | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}%" if x_val is not None else ""
                print(f"Epoch {epoch:4d} | Loss: {full_train_loss:.4f} | Acc: {full_train_acc:.2f}%{val_info}")


    
    def accuracy(self, y_pred, y_true):
        y_pred = (y_pred >= 0.5).astype(int)
        return np.mean(y_pred == y_true) * 100
    

    def predict(self, X):
        """Make predictions"""
        y_pred = self.mlp.forward(X)
        return (y_pred >= 0.5).astype(int)
    
    def predict_proba(self, X):
        """Predict probabilities"""
        return self.mlp.forward(X)


In [44]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_moons, make_circles
from sklearn.model_selection import train_test_split

# Generate datasets
print("📊 GENERATING DATASETS...")
np.random.seed(42)

# Moons dataset
X_moons, y_moons = make_moons(n_samples=500, noise=0.1, random_state=42)
y_moons = y_moons.reshape(-1, 1)

# Circles dataset  
X_circles, y_circles = make_circles(n_samples=500, noise=0.1, factor=0.5, random_state=42)
y_circles = y_circles.reshape(-1, 1)

print(f"Moons dataset: X {X_moons.shape}, y {y_moons.shape}")
print(f"Circles dataset: X {X_circles.shape}, y {y_circles.shape}")

# Split into train/validation sets
X_moons_train, X_moons_val, y_moons_train, y_moons_val = train_test_split(
    X_moons, y_moons, test_size=0.2, random_state=42
)

X_circles_train, X_circles_val, y_circles_train, y_circles_val = train_test_split(
    X_circles, y_circles, test_size=0.2, random_state=42
)

print(f"\nTrain/Validation splits:")
print(f"Moons - Train: {X_moons_train.shape}, Val: {X_moons_val.shape}")
print(f"Circles - Train: {X_circles_train.shape}, Val: {X_circles_val.shape}")


print("\n" + "="*60)
print("🌙 TRAINING ON MOONS DATASET")
print("="*60)

# Create and train neural network for moons
nn_moons = NeruronNetworkLearning(
    layer_size=[2, 16, 8, 1],  # Input: 2, Hidden: 16→8, Output: 1
    activations=['Relu', 'Relu', 'Sigmoid'],
    Lr=0.1,
    batch_size=32
)

print("🧠 Neural Network Architecture for Moons:")
for i, layer in enumerate(nn_moons.mlp.layers):
    print(f"Layer {i}: {layer.w.shape[0]} → {layer.w.shape[1]} neurons ({layer.activation})")

# Train the model
nn_moons.train(
    X_moons_train, y_moons_train,
    x_val=X_moons_val, y_val=y_moons_val,
    epochs=1000,
    verbose=True
)

# Final evaluation
final_pred_moons = nn_moons.predict(X_moons_val)
final_proba_moons = nn_moons.predict_proba(X_moons_val)
final_acc_moons = nn_moons.accuracy(final_proba_moons, y_moons_val)

print(f"\n🎯 Final Moons Validation Accuracy: {final_acc_moons:.2f}%")

📊 GENERATING DATASETS...
Moons dataset: X (500, 2), y (500, 1)
Circles dataset: X (500, 2), y (500, 1)

Train/Validation splits:
Moons - Train: (400, 2), Val: (100, 2)
Circles - Train: (400, 2), Val: (100, 2)

🌙 TRAINING ON MOONS DATASET
🧠 Neural Network Architecture for Moons:
Layer 0: 2 → 16 neurons (Relu)
Layer 1: 16 → 8 neurons (Relu)
Layer 2: 8 → 1 neurons (Sigmoid)
Epoch    0 | Loss: 0.6912 | Acc: 51.75% | Val Loss: 0.6927 | Val Acc: 43.00%
Epoch  100 | Loss: 0.0587 | Acc: 98.00% | Val Loss: 0.0664 | Val Acc: 98.00%
Epoch  200 | Loss: 0.0071 | Acc: 100.00% | Val Loss: 0.0111 | Val Acc: 100.00%
Epoch  300 | Loss: 0.0031 | Acc: 100.00% | Val Loss: 0.0052 | Val Acc: 100.00%
Epoch  400 | Loss: 0.0021 | Acc: 100.00% | Val Loss: 0.0049 | Val Acc: 100.00%
Epoch  500 | Loss: 0.0014 | Acc: 100.00% | Val Loss: 0.0034 | Val Acc: 100.00%
Epoch  600 | Loss: 0.0011 | Acc: 100.00% | Val Loss: 0.0030 | Val Acc: 100.00%
Epoch  700 | Loss: 0.0008 | Acc: 100.00% | Val Loss: 0.0024 | Val Acc: 100.00

In [ ]:
#Breast Cancer Dataset
from sklearn.datasets import load_breast_cancer
import pandas as pd

def data_loading(Dataset):
    #exploring the data
    bcd = load_breast_cancer()
    #print(bcd.keys())

    X = pd.DataFrame(data = bcd.data)
    #X = bcd.data
    print(X.shape[1])
    Y = pd.DataFrame(data = bcd.target)
    #print(Y)
    #print(X.info)
    return X, Y

def train_test_split(Dataset):


In [ ]:
#the initionlizer class
class WInitializationfunctions:

    #shape is a matric that has the diemntions input and output value
    
    def zeros(shape):
        return np.zeros(shape)
    
    def random(shape, scale = 0.01):
        return np.random.randn(*shape) * scale
    
    def xavier(shape):
        n_in, o_in = shape[0], shape[1]
        scale = np.sqrt(2.0 / (n_in + o_in))
        return np.random.randn(*shape) * scale
    
    def he(shape):
        n_in = shape[0]
        scale = np.sqrt(2.0 / n_in)
        return np.random.randn(*shape) * scale

In [ ]:
class DDesnsLayer:

    #n_output mean the number of neurons per layer
    def __init__(self, n_inputs, n_outputs, activation, Lr = 0.01):
        self.w = np.random.randn(n_inputs, n_outputs) * 0.1 # here we multiplied by 0.1 to avoid the explding of the gradients because if the values of w came large then z will be big
        self.b = np.zeros((1, n_outputs))
        self.activation = activation
        self.Lr = Lr
        self.lose =[] #for viz
        self.n_inputs = n_inputs
        self.n_outputs = n_outputs

    #activation functions
    def sigmoid(self, X):
        return 1 / (1 + np.exp(-np.clip(X, -250, 250)))
    
    def relu(self, X):
        return np.maximum(0, X)
    
    def tanh(self, X):
        return np.tanh(X)

    def forward(self, X):
        self.X = X
        self.z = X @ self.w + self.b
        #print(z)

        #calling activation functions to activate the neuron to produce output(prediction)
        if self.activation == "Linear":
            self.y_pred = self.z
        elif self.activation == "Relu":
            self.y_pred = self.relu(self.z)
        elif self.activation == "Tanh":
            self.y_pred = self.tanh(self.z)
        elif self.activation == "Sigmoid":
            self.y_pred = self.sigmoid(self.z)

        return self.y_pred
    

    def loss(self, los, y_pred, Y_true):
        dL_dy = 0
        if los == "MSE":
            L = np.mean((Y_true - y_pred) ** 2)
            self.lose.append(L)
            n = Y_true.shape[0]
            dL_dy = (2 /n) * (Y_true - y_pred)
        elif los == "BCE":
            # use the clip
            epsilon = 1e-15
            y_pred = np.clip(y_pred, epsilon, 1 - epsilon) #to avoid log(0)
            L = - np.mean(Y_true * np.log(y_pred) + (1 - Y_true) * np.log(1 - y_pred))
            self.lose.append(L)
            dL_dy = (y_pred - Y_true) / Y_true.shape[0]
            #(y_pred * (1- y_pred) * len(Y_true))
        
        return dL_dy


    # backword --> calculate gradients
    def backword(self, dL_dy):

        # activations derivatieves
        dy_dz = 0
        if self.activation == "Linear":
            dy_dz = 1
        elif self.activation == "Relu":
            dy_dz = (self.z > 0).astype(float)
        elif self.activation == "Tanh":
            dy_dz = 1 - self.y_pred**2
        elif self.activation == "Sigmoid":
            dy_dz = self.y_pred * (1 - self.y_pred)
        
        dL_dz = dL_dy * dy_dz
        dL_dw = self.X.T @ dL_dz
        dL_db = np.sum(dL_dz, axis=0, keepdims=1) * 1
        dL_dx = dL_dz @ self.w.T

        return dL_dw, dL_db, dL_dx

    def step(self, dL_dw, dL_db):
        #update parameters
        self.w = self.w - self.Lr * dL_dw
        self.b = self.b - self.Lr * dL_db

        return self.w, self.b
 

    def viz(self):
        plt.figure(figsize=(10, 6))
        plt.title("🧠 Neural Layer Architecture", fontsize=14, pad=20)

        # Draw neurons
        for i in range(self.n_inputs):  # Input layer
            plt.scatter(0, i, s=500, c='lightblue', edgecolors='black', zorder=5)
            plt.text(0, i, f'x{i+1}', ha='center', va='center', fontweight='bold')
            
        for j in range(self.n_outputs):  # Output layer
            plt.scatter(2, j, s=500, c='lightgreen', edgecolors='black', zorder=5)
            plt.text(2, j, f'a{j+1}', ha='center', va='center', fontweight='bold')
            
            # Draw connections
            for i in range(self.n_inputs):
                plt.plot([0, 2], [i, j], 'gray', alpha=0.3)

        plt.xlim(-0.5, 2.5)
        plt.ylim(-0.5, 4.5)
        plt.axis('off')
        plt.text(0, -0.3, f"Input Layer {self.n_inputs}, neurons", ha='center', fontsize=12)
        plt.text(2, -0.3, f"Dense Layer {self.n_outputs}, neurons", ha='center', fontsize=12)
        plt.show()

